In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pickle
import normalize_data
import random
import train
import torch
import torch.nn as nn
from dataset import Dataset, to_device
from model import NNBig
from time import time
from tqdm.notebook import tqdm

pd.set_option('display.max_columns', 999)
pd.set_option('display.precision', 10)

In [2]:
def get_ref_X_y(df):
    ref_cols = [c for c in df.columns if c.startswith('__')]
    X_cols = [c for c in df.columns if c.startswith('X_')]
    y_cols = [c for c in df.columns if c.startswith('y_')]
    return (df[ref_cols], df[X_cols], df[y_cols])

In [14]:
raw_data = {} # loads raw data and stores as a dict cache

def dataset_key(dataset='', validation=False):
    return dataset+('test' if validation else 'train')


def load_data(raw, dataset='', validation=False):
    '''
    Return dataframe matching data set and validation. Dictionary input will be updated.

    Parameters
    ----------
    raw : dict
        dictionary which caches the dataframes and will be updated accordingly

    dataset : str
        which dataset to use? valid input includes: empty str for full set, sample_, and secret_

    validation : bool
        load validation set? if true then use _test, otherwise use _train.  Note secret_ doesn't have _train
    '''
    key = dataset+('test' if validation else 'train')
    if key not in raw:
        print(f"Loading data to cache for: {key}")
        raw[key] = pd.read_pickle(f'{os.environ["GP_HIST_PATH"]}/../t4_data/{key}.pkl')
    return raw[key]

In [26]:
configurations = {
    'dataset' : '', # '', 'sample_', 'secret_'
    'model_identifier' : "GOLD_FULL",
    'model_path' : f"{os.environ['GP_HIST_PATH']}/../t4_models",
    'device' : 'cpu',
    'random_seed' : 0,
    'max_epochs' : 50000,
    'do_validate' : True,
    'model_definition' : [ # this is only used to control the base and feature sequential layers
        ('layer', 1000), ('relu', None),
        ('layer', 500), ('relu', None),
    ],
    'feature_head_out' : 100,
    'train_params' : {
        'batch_size': 200000,
        'shuffle': True,
        'num_workers': 3,
        'pin_memory': True,
    },
    'test_params' : {
        'batch_size': 200000,
        'num_workers': 3,
        'pin_memory': True,
    },
}

In [27]:
%%time

train_df = normalize_data.normalize_all_columns(load_data(raw_data,dataset=configurations['dataset'],validation=False))
test_df = normalize_data.normalize_all_columns(load_data(raw_data,dataset=configurations['dataset'],validation=True))

train_df = train_df.dropna()
test_df = test_df.dropna()
# train_df = train_df.sample(100000, random_state=42)
# test_df = test_df.sample(10000, random_state=42)

CPU times: user 1min 7s, sys: 39.4 s, total: 1min 46s
Wall time: 1min 15s


In [29]:
ref_train, X_train, y_train = get_ref_X_y(train_df)
ref_test, X_test, y_test = get_ref_X_y(test_df)

In [30]:
y_cols = ['y_INCLINATION', 'y_ECCENTRICITY', 'y_MEAN_MOTION', 'y_RA_OF_ASC_NODE_REG', 'y_RA_OF_ASC_NODE', 'y_ARG_OF_PERICENTER_REG', 'y_ARG_OF_PERICENTER', 'y_BSTAR', 'y_REV_MA_REG', 'y_MEAN_ANOMALY']
y_cols = ['y_INCLINATION', 'y_ECCENTRICITY', 'y_MEAN_MOTION', 'y_RA_OF_ASC_NODE_REG', 'y_ARG_OF_PERICENTER_REG', 'y_REV_MA_REG']
y_cols = ['y_RA_OF_ASC_NODE_REG']
y_train = y_train[y_cols]
y_test = y_test[y_cols]

In [31]:
path = configurations.get('model_path', None)
torch.manual_seed(configurations.get('random_seed',0))
device = configurations.get('device','cpu')
pyt_device = torch.device(device)

training_set = Dataset(X_train, y_train)
training_generator = torch.utils.data.DataLoader(training_set, **configurations['train_params'])
testing_set = Dataset(X_test, y_test)
testing_generator = torch.utils.data.DataLoader(testing_set, **configurations['test_params'])

full_net, loss_func, optimizer, mean_losses, next_epoch, = train.load_model_with_config(configurations, training_set, False)

net = full_net.raan_model

Loading existing model


In [32]:
print(optimizer)

AdamW (
Parameter Group 0
    amsgrad: True
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.001
)


In [33]:
optimizer = torch.optim.AdamW(net.parameters(), lr=0.00001, weight_decay=0.0001, amsgrad=True)

In [35]:
to_device(net, pyt_device)
net.train()
#     print(net)

if next_epoch == configurations['max_epochs']:
    print("Model finished training. To retrain set force_train = True ")
    net.eval()
else:
    epbar = tqdm(range(next_epoch, configurations['max_epochs']))
    for epoch in epbar:
        epbar.set_description(f"Epoch {epoch+1}")

        running_eloss = 0
        running_vloss = 0

        ipbar = tqdm(training_generator, leave=False)
        ipbar.set_description(f"Training")

        for i, (x, y) in enumerate(ipbar):
            x = to_device(x, pyt_device)
            y = to_device(y, pyt_device)

            optimizer.zero_grad()
            prediction = net(x)     # input x and predict based on x
            loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients
            running_eloss += loss.item()

        net.eval()
        mean_vlosses = 0
#         if configurations['do_validate']:
#             with torch.set_grad_enabled(False):
#                 vpbar = tqdm(testing_generator, leave=False)
#                 vpbar.set_description("Validating")
#                 for i, (x, y) in enumerate(vpbar):
#                     x = to_device(x, pyt_device)
#                     y = to_device(y, pyt_device)
#                     prediction = net(x)
#                     loss = loss_func(prediction, y)
#                     running_vloss += loss.item()
#             mean_vlosses = running_vloss / len(testing_generator)

        mean_elosses = running_eloss / len(training_generator)
        mean_losses.append((mean_elosses, mean_vlosses))
        
        full_net.raan_model = net # save the full_net back
        
        train.save_model_with_config(configurations, net=full_net, loss_func=loss_func, optimizer=optimizer,
                               mean_losses=mean_losses, next_epoch=epoch+1,
                              )
        net.train()

        epbar.set_postfix({'train_loss':f"{mean_elosses:.9f}", 'val_loss':f"{mean_vlosses:.9f}"})
    net.eval()

  0%|          | 0/47185 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():    
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():    
assert self

  0%|          | 0/498 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>
Traceback (most recent call last):
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>
Traceback (most recent call last):
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers

  0%|          | 0/498 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0><function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
        if w.is_alive():if w.is_alive():
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/

  0%|          | 0/498 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>
Traceback (most recent call last):
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fd69fb051f0>  File "/data1/home/ttcchen/anaconda3/envs/siads-orbital/lib/python3.8/site-p

KeyboardInterrupt: 

In [36]:
train.save_model_with_config(configurations, net=full_net, loss_func=loss_func, optimizer=optimizer,
                               mean_losses=mean_losses, next_epoch=epoch+1,
                              )

In [ ]:
model, _, _, mean_losses, _ = train.load_model_with_config(configurations)

tl, vl = zip(*mean_losses)

fig,ax = plt.subplots()
ax.plot(tl, label="Training Loss")
ax.plot(vl, label="Validation Loss")

fig.legend()
plt.show()

In [ ]:
trained_model = model

In [ ]:
sample_idx = random.sample(list(X_test.index), 10000)

X_sample = X_test.loc[sample_idx]
y_sample = y_test.loc[sample_idx]

In [ ]:
# y_train_pred = train.predict(trained_model, X_train, y_train, device="cpu") # get predictions for each train
# y_train_pred_df = pd.DataFrame(y_train_pred, columns=y_train.columns)  # put results into a dataframe
y_sample_pred = train.predict(trained_model, X_sample, y_sample, device="cpu") # get predictions for each train
y_sample_pred_df = pd.DataFrame(y_sample_pred, columns=y_sample.columns)  # put results into a dataframe

In [ ]:
print(f'    Test set MAE (L1) loss: {mean_absolute_error(y_sample, y_sample_pred_df)}')
print(f'    Test set MSE (L2) loss: {mean_squared_error(y_sample, y_sample_pred_df)}')

# random.seed(342798)
sample = random.sample(list(y_sample_pred_df.index), 10)
# sample = [0,1]

print("Test - Ground Truth (normalized):")
display(y_sample)
print("Test - Prediction (normalized):")
display(y_sample_pred_df)
print("Ground Truth Diffs from X_:")
display(y_sample - X_sample[['X_INCLINATION_1','X_ECCENTRICITY_1','X_MEAN_MOTION_1']].values)
print("Pred Diffs from X_:")
display(y_sample_pred_df - X_sample[['X_INCLINATION_1','X_ECCENTRICITY_1','X_MEAN_MOTION_1']].values)


In [ ]:
# print(f'    Train set MAE (L1) loss: {mean_absolute_error(y_train, y_train_pred_df)}')
# print(f'    Train set MSE (L2) loss: {mean_squared_error(y_train, y_train_pred_df)}')

# random.seed(0)
# sample = random.sample(list(y_train_pred_df.index), 10)

# print("Train - Ground Truth (normalized):")
# display(y_train.loc[sample])
# # print("Train - Ground Truth (non-normalized):")
# # display(normalize_data.normalize_all_columns(y_train.iloc[:,3:].loc[sample].copy(), reverse=True))  # see ground truths
# print("Train - Prediction (normalized):")
# display(y_train_pred_df.loc[sample])
# # print("Train - Prediction (non-normalized):")
# # display(normalize_data.normalize_all_columns(y_train_pred_df.loc[sample].copy(), reverse=True))  # See predictions

In [ ]:
# errors = y_train.loc[sample].iloc[:,3:] - y_train_pred_df.loc[sample]

# display(errors)

# display(errors.std())

# display(y_train.loc[sample].iloc[:,3:])
# display(y_train_pred_df.loc[sample])

In [ ]:
# def row_to_compare(X, y, y_pred, row):
#     epoch = X.iloc[row].EPOCH_y
#     X0 = clean_data.normalize_all_columns(X_train.iloc[row].copy(), reverse=True)
#     y0 = clean_data.normalize_all_columns(y_train.iloc[row].copy(), reverse=True)
#     y1 = clean_data.normalize_all_columns(y_train_pred_df.iloc[row].copy(), reverse=True)

#     # Ground truth
#     y0_xyz = clean_data.get_satellite_xyz(bst=0.0001,
#                                           ecc=y0.ECCENTRICITY,
#                                           aop=y0.ARG_OF_PERICENTER,
#                                           inc=y0.INCLINATION,
#                                           mea=y0.MEAN_ANOMALY,
#                                           mem=y0.MEAN_MOTION,
#                                           raa=y0.RA_OF_ASC_NODE,
#                                           epoch=epoch,)
#     # Prediction
#     y1_xyz = clean_data.get_satellite_xyz(bst=0.0001,
#                                           ecc=y1.ECCENTRICITY,
#                                           aop=y1.ARG_OF_PERICENTER,
#                                           inc=y1.INCLINATION,
#                                           mea=y1.MEAN_ANOMALY,
#                                           mem=y1.MEAN_MOTION,
#                                           raa=y1.RA_OF_ASC_NODE,
#                                           epoch=epoch,)
#     # Propigation
#     y2_xyz = clean_data.get_satellite_xyz(bst=X0.BSTAR,
#                                           ecc=X0.ECCENTRICITY,
#                                           aop=X0.ARG_OF_PERICENTER,
#                                           inc=X0.INCLINATION,
#                                           mea=X0.MEAN_ANOMALY,
#                                           mem=X0.MEAN_MOTION,
#                                           raa=X0.RA_OF_ASC_NODE,
#                                           epoch=epoch,)
#     print(f'Ground Truth: {y0_xyz}')
#     print(f'Predicted: {y1_xyz}')
#     print(f'Propigation: {y2_xyz}')
    
#     print (f'Prediction Error: {sum((y1_xyz-y0_xyz)**2)**0.5} km')
#     print (f'Propigation Error: {sum((y2_xyz-y0_xyz)**2)**0.5} km')
    

# for row in range(2):
#     print (f'Row {row}:')
#     row_to_compare(X_train, y_train, y_train_pred_df, row)
#     print('')